In [1]:
import pandas as pd
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
df1 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df2 = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
# Q1: Read the data for January. How many columns are there?

print("Q1, Answer: ", df1.shape[1])

Q1, Answer:  19


In [4]:
# Q2: What's the standard deviation of the trips duration in January?

df1['duration'] = df1.tpep_dropoff_datetime - df1.tpep_pickup_datetime
df1.duration = df1.duration.apply(lambda td: td.total_seconds() / 60)

print("Q2, Answer: ", round(df1['duration'].std(), 2))

Q2, Answer:  42.59


In [5]:
# Q3: What fraction of the records left after you dropped the outliers?

df = df1[(df1.duration >= 1) & (df1.duration <= 60)]

print("Q3, Answer: ", round(len(df)/len(df1), 2))

Q3, Answer:  0.98


In [6]:
# Q4: Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
df = df[categorical]

df_dicts = df.to_dict(orient='records')
dv = DictVectorizer()

X = dv.fit_transform(df_dicts)
print("Q4, Answer: ", X.shape[1])

/tmp/ipykernel_36942/543766114.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


Q4, Answer:  515


In [7]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df = df[['PULocationID', 'DOLocationID', 'duration']]
    
    return df

In [8]:

# Q5: Training a model
# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

train_dicts = df_train[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer()

X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print("Q5, Answer: ", round(mean_squared_error(y_train, y_pred, squared=False), 2))

Q5, Answer:  7.65


In [9]:
# Q6 Evaluating the model
# Now let's apply this model to the validation dataset (February 2023).

# What's the RMSE on validation?

df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')


X_val = dv.transform(val_dicts)

target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

print("Q6, Answer: ", round(mean_squared_error(y_val, y_pred, squared=False), 2))

Q6, Answer:  7.81
